In [1]:
# !pip install ctransformers[cuda]
# !pip3 install numpy==1.26.4
import numpy as np
np.__version__

# restart after

'1.26.3'

In [2]:
import os
# from dotenv import load_dotenv
# load_dotenv('.env')
# hf_api = os.getenv('HF_API')

In [3]:
!pip install transformers==4.38.0
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl
!pip install unstructured ragatouille
# reranker
from ragatouille import RAGPretrainedModel
!nvcc --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 78.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.7 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.3 which is incompatible.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 2.9 MB/s eta 0:00:00
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
# this will take time :c
!CUDACXX=/usr/local/cuda-12.1/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 178.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 296.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 237.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 357.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 349.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_26_x86_64.whl size=23199999 sha256=a13e70f083f1c9fcf5f170e29d7bdb3e3a05a89e7d7bbee66811d4ac1e2c14f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqvjlzl6/wheels/e5/09/9d/c413053f6258cb2546cc792418c595e276f9efd5db31a80377
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
  

In [6]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
import pandas as pd

In [8]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [9]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

In [10]:
# pip list --outdated

# Specify the model/versions, etc.
## also specify the knowledge base file and the reference answer directory

In [11]:
EMBEDDING_MODEL = "thenlper/gte-base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
!pwd

/home/ec2-user/SageMaker/srg-rag-final


In [14]:
FAISS_FILE = '../faiss_index_total_final'
# num_retrieved_docs: int = 5
# num_docs_final: int = 3
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(FAISS_FILE, embedding_model, allow_dangerous_deserialization = True)

In [15]:
model_path = "TheBloke/Llama-2-7B-Chat-GGUF"
model_basename = "llama-2-7b-chat.Q4_K_M.gguf"
model_path = hf_hub_download(repo_id=model_path, filename=model_basename)

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

# Reader LLM

In [16]:
# https://www.reddit.com/r/LocalLLaMA/comments/1343bgz/what_model_parameters_is_everyone_using/
# temperature	0.7
# repetition_penalty=	1.176
# top_k	= 40
# top_p= 0.1

# Initialize Large Language Model for answer generation
llm_answer_gen = LlamaCpp(
streaming = True,
model_path = model_path,
temperature=.2,
n_gpu_layers=-1,
top_p=.1,
top_k	= 40,
repetition_penalty=	1.176,
verbose=True,
n_ctx=2400
)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! repetition_penalty is not default parameter.
                repetition_penalty was transferred to model_kwargs.
                Please confirm that repetition_penalty is what you intended.
  warnings.warn(
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv

In [17]:
llm_answer_gen.verbose = False

# RAG Function

In [18]:
# link: https://python.langchain.com/docs/use_cases/chatbots/retrieval

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# https://stackoverflow.com/questions/76551067/how-to-create-a-langchain-doc-from-an-str

def answer_llama_new(
    question: str,
    knowledge_index: FAISS,
    reranker: RAGPretrainedModel,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 3,
    llm: LlamaCpp = llm_answer_gen,
    ): #-> Tuple[str, List[LangchainDocument]]

    print("=> Retrieving documents...")
    relevant_docs_acquired = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_doc_id = [doc.metadata['source'] for doc in relevant_docs_acquired]

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

        relevant_docs = [LangchainDocument(page_content=doc["content"],
                                           metadata={'source': relevant_doc_id[ doc['result_index']] } ) for doc in relevant_docs]

        relevant_docs_content = [doc.page_content for doc in relevant_docs]
        relevant_doc_id = [doc.metadata for doc in relevant_docs]

    else:
        relevant_docs_content = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = relevant_docs_acquired

    relevant_docs_content = relevant_docs_content[:num_docs_final]
    relevant_docs_indexed = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs_content)])

    context_and_question = f"Please provide concise answers. Extract the most relevant information from the given context while keeping the responses as short as possible. Include multiple correct answers if necessary. If no relevant context, then say 'I dont know'.\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAgain we just want main key points in the answers. For example, if the question is 'What is the biggest planet in the solar system?'; the answer should be 'Jupiter'\n.Answer:"
    
    answer = llm.invoke(context_and_question)
    
    if answer is not None:
        new_answer = answer.split(': ', 1)
        if len(new_answer) > 1:
            answer = new_answer[1]
        else:
            answer = new_answer[0]
    # print(f"Question: {question}")
    
    forbidden_list = ["According to the provided context, ", "Based on the provided context,", '\n']
    for phrase in forbidden_list:
        answer = answer.replace(phrase, " ")
    answer = answer.replace("<pad>", "").replace("</s>", "").replace("\n", "").strip()
    # print(f"Answer: {answer}")
    
    return answer, relevant_docs_content

## Evaluation

In [19]:
# specify output csv 
input_file = 'qa_gold-llama-1-shot-no_chat_template.csv'
output_file = 'llama-1-shot-no_chat_template-output.csv'
csv_input_dir = f'csv_qa_gold/{input_file}'
csv_output_dir = f'csv_qa_gold/{output_file}'

In [20]:
def initialize_model_answer_column(df):
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = None
    return df

In [21]:
import glob
import os
import pandas as pd

directory_csv = 'csv_qa_gold'
csv_files = ['csv_qa_gold/test_combined.csv']

# read in the csv files in the directory and concatenate
df_total = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    print(df.shape)
    df_total = pd.concat([df_total, df], axis=0)
print(df_total.shape)
df_total = initialize_model_answer_column(df_total)
df_total.to_csv(csv_input_dir, index=False) 
# doing this so the original is untouched

(191, 3)
(191, 3)


In [22]:
# df_total.iloc[10:20, :] # please check that excel does not fuck up for row 15,16 the November 2006 to Nov-06 or some other format

In [23]:
def generate_answer(question,llm, return_relevant_docs = False):
    answer, relevant_docs = answer_llama_new(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = llm)
    if return_relevant_docs:
        return answer, relevant_docs
    return answer

In [24]:
# Function to run over a list of questions and return the answers
# Define the function to process the CSV and add model answers

def generate_answers_and_save(csv_path, llm):
    df = pd.read_csv(csv_path)
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = ''
    for index, row in df.iterrows():
        ModelAnswer = generate_answer(row['Question'],llm=llm)
        df.at[index, 'ModelAnswer'] = ModelAnswer
        df.to_csv(csv_output_dir, index=False)
    return df

In [25]:
df_with_model_answers = generate_answers_and_save(csv_path=csv_input_dir, llm = llm_answer_gen)
print(df_with_model_answers.shape)
df_with_model_answers.head()

=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 43.69it/s]

llama_print_timings:        load time =    1215.36 ms
llama_print_timings:      sample time =       7.17 ms /    12 runs   (    0.60 ms per token,  1672.94 tokens per second)
llama_print_timings: prompt eval time =   10044.01 ms /   979 tokens (   10.26 ms per token,    97.47 tokens per second)
llama_print_timings:        eval time =     272.85 ms /    11 runs   (   24.80 ms per token,    40.32 tokens per second)
llama_print_timings:       total time =   10603.53 ms /   990 tokens


=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 236.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 30.24it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1215.36 ms
llama_print_timings:      sample time =      13.81 ms /    23 runs   (    0.60 ms per token,  1665.82 tokens per second)
llama_print_timings: prompt eval time =    9807.40 ms /  1024 tokens (    9.58 ms per token,   104.41 tokens per second)
llama_print_timings:        eval time =     576.37 ms /    22 runs   (   26.20 ms per token,    38.17 tokens per second)
llama_print_timings:       total time =   10712.94 ms /  1046 tokens


=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 236.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


...

100%|██████████| 1/1 [00:00<00:00, 84.65it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1215.36 ms
llama_print_timings:      sample time =      10.18 ms /    17 runs   (    0.60 ms per token,  1670.11 tokens per second)
llama_print_timings: prompt eval time =    1660.92 ms /   187 tokens (    8.88 ms per token,   112.59 tokens per second)
llama_print_timings:        eval time =     362.48 ms /    16 runs   (   22.66 ms per token,    44.14 tokens per second)
llama_print_timings:       total time =    2127.38 ms /   203 tokens


=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 85.08it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1215.36 ms
llama_print_timings:      sample time =      55.37 ms /    92 runs   (    0.60 ms per token,  1661.52 tokens per second)
llama_print_timings: prompt eval time =    2195.73 ms /   248 tokens (    8.85 ms per token,   112.95 tokens per second)
llama_print_timings:        eval time =    2058.25 ms /    91 runs   (   22.62 ms per token,    44.21 tokens per second)
llama_print_timings:       total time =    4625.13 ms /   339 tokens


(191, 4)


,Category,Question,Answer,ModelAnswer
0,webpages,Do students in Kiltie have to memorize music?,No,"No, students do not have to memorize music."
1,webpages,Do I need to email my home address to the Kiltie Band Director if I'm interested in joining?,Yes,"Yes, you should email your home address to the Kiltie Band Director if you are interested in joining."
2,webpages,Do I need to audition to join the Kiltie Band?,No,"No, any member of the campus community with music experience is able to join the Kiltie Band!"
3,webpages,Where does the Kiltie Band rehearse?,CUC Studio Theater,The Kiltie Band rehearses in the CUC Studio Theater.
4,webpages,When should I get an instrument from the Kiltie Band if I need one?,4:30p.m. before the first rehearsal,The Kiltie Band has instruments available for borrowing starting at 4:30 p.m. before the first rehearsal on Monday.


In [27]:
# with additional extraction prompt after generation without prompt
type(df_with_model_answers['ModelAnswer'][0]),type(df_with_model_answers['Answer'][0])
df_with_model_answers = df_with_model_answers.astype(str)  # Convert columns to string type

from evaluation_csv import total_score_csv
exact_match, f1_score, recall_score = total_score_csv(df_with_model_answers['ModelAnswer'], df_with_model_answers['Answer'])
print(f'Exact match: {exact_match}')
print(f'F1 score: {f1_score}')
print(f'Recall score: {recall_score}')

Exact match: 0.08900523560209424
F1 score: 0.2469942301836472
Recall score: 0.48594194350185055


In [28]:
for category in df_with_model_answers['Category'].unique():
    print(f'Category: {category}')
    df_category = df_with_model_answers[df_with_model_answers['Category'] == category]
    exact_match, f1_score, recall_score = total_score_csv(df_category['ModelAnswer'], df_category['Answer'])
    print(f'Exact match: {exact_match}')
    print(f'F1 score: {f1_score}')
    print(f'Recall score: {recall_score}')
    print('\n')

Category: webpages
Exact match: 0.07407407407407407
F1 score: 0.2991502362824957
Recall score: 0.6361828568334417


Category: tabular_webpages
Exact match: 0.0
F1 score: 0.09635171805447396
Recall score: 0.44166666666666665


Category: other_pdf
Exact match: 0.0
F1 score: 0.154057882798292
Recall score: 0.2581262939958592


Category: papers_pdf
Exact match: 0.0
F1 score: 0.038841342988808425
Recall score: 0.07936507936507937


Category: schedule_pdf
Exact match: 0.0
F1 score: 0.12626262626262627
Recall score: 0.1212121212121212


Category: jsons
Exact match: 0.18571428571428572
F1 score: 0.34399769336092967
Recall score: 0.6141290166834544


Category: json_hard
Exact match: 0.0
F1 score: 0.019620958751393535
Recall score: 0.05795454545454546




In [29]:
df_with_model_answers.to_csv('../llama-no-temp-one-shot-output.csv', index=False)

In [30]:
display(df_with_model_answers.head(60))

,Category,Question,Answer,ModelAnswer
0,webpages,Do students in Kiltie have to memorize music?,No,"No, students do not have to memorize music."
1,webpages,Do I need to email my home address to the Kiltie Band Director if I'm interested in joining?,Yes,"Yes, you should email your home address to the Kiltie Band Director if you are interested in joining."
2,webpages,Do I need to audition to join the Kiltie Band?,No,"No, any member of the campus community with music experience is able to join the Kiltie Band!"
3,webpages,Where does the Kiltie Band rehearse?,CUC Studio Theater,The Kiltie Band rehearses in the CUC Studio Theater.
4,webpages,When should I get an instrument from the Kiltie Band if I need one?,4:30p.m. before the first rehearsal,The Kiltie Band has instruments available for borrowing starting at 4:30 p.m. before the first rehearsal on Monday.
5,webpages,When is Andrew Carnegie's birthday?,"November 25th, 1835","Andrew Carnegie's birthday is on November 25, 1835."
6,webpages,What time are rehearsals on Game Days for the Kiltie Band?,9:30-11:00 a.m.,Rehearsals on Game Days for the Kiltie Band are at 9:30-11:00 a.m.
7,webpages,How should Kilties promote the success of the Kiltie Band?,"attendance, attitude, and dependabiliy","By attending rehearsals and performances, being on time and prepared, and contributing to a positive and supportive atmosphere."
8,webpages,What should I email to the band director if I'm interested in joining the Kiltie Band?,"name, high school, address at Carnegie Mellon, home address, cell number, home number, whether you plan on being in the band or colorguard, if in the band what instrument(s) you play, and whether you need to borrow an instrument or equipment",The email address to send your interest in joining the Kiltie Band to is jolisar@andrew.cmu.edu .
9,webpages,What university is the Kiltie Band a part of?,Carnegie Mellon,Carnegie Mellon University
